In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Aug 18 03:26:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/
!rm -rf cloth-segmentation
!git clone https://github.com/levindabhi/cloth-segmentation.git
%cd cloth-segmentation
!gdown --id 1mhF3yqd7R-Uje092eypktNl-RoZNuiCJ
!mkdir input_images
!mkdir output_images

/content
Cloning into 'cloth-segmentation'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 62 (delta 3), reused 6 (delta 0), pack-reused 52
Unpacking objects: 100% (62/62), done.
/content/cloth-segmentation
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1mhF3yqd7R-Uje092eypktNl-RoZNuiCJ
To: /content/cloth-segmentation/cloth_segm_u2net_latest.pth
100% 177M/177M [00:00<00:00, 216MB/s]


/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Chanel/Fall Couture Collection  
/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Chanel/Fall Ready to Wear Collection  

/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Chanel/Spring Couture Collection  

/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Chanel/Spring Ready to Wear Collection

/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Christian Dior/Fall Couture Collection  
/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Christian Dior/Fall Ready to Wear Collection  
/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Christian Dior/Spring Couture Collection  
/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Christian Dior/Spring Ready to Wear Collection

# Upload input images in *input_images* folder.



In [ ]:
!rm -rf output_images
!mkdir output_images
import os
from tqdm import tqdm
# from tqdm.notebook import tqdm
from PIL import Image
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

from data.base_dataset import Normalize_image
from utils.saving_utils import load_checkpoint_mgpu

from networks import U2NET

assert torch.cuda.is_available()
device = 'cuda'

###################################
image_dir = '/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Christian Dior/Spring Couture Collection'
result_dir = 'output_images/'
checkpoint_path = 'cloth_segm_u2net_latest.pth'

transforms_list = []
transforms_list += [transforms.ToTensor()]
transforms_list += [Normalize_image(0.5, 0.5)]
transform_rgb = transforms.Compose(transforms_list)

net = U2NET(in_ch=3, out_ch=4)
net = load_checkpoint_mgpu(net, checkpoint_path)
net = net.to(device)
net = net.eval()

!rm -rf input_images/.ipynb_checkpoints
!rm -rf '/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Chanel/Spring Couture Collection/.ipynb_checkpoints'
### edited by jemu
dirs = sorted(os.listdir(image_dir))

# '0001, 0002, 0003 .... ' for file nameing
cnt = []
for i in range(10000):
  cnt.append(f'{i:04d}')
cnt = iter(cnt)

for dir in tqdm(dirs):
  images_list = sorted(os.listdir(os.path.join(image_dir, dir)))
  year = str([i for i in dir.split() if i.isnumeric()][0])
  for image_name in images_list:
    if image_name[-4:] != ".jpg": continue
    img = Image.open(os.path.join(image_dir, dir, image_name)).convert('RGB')
    img_size = img.size
    img = img.resize((768, 768), Image.BICUBIC)
    image_tensor = transform_rgb(img)

    # edited
    masked_img = transforms.Compose([transforms.ToTensor()])(img).cpu().numpy() 

    image_tensor = torch.unsqueeze(image_tensor, 0)
    
    output_tensor = net(image_tensor.to(device))
    output_tensor = F.log_softmax(output_tensor[0], dim=1)
    output_tensor = torch.max(output_tensor, dim=1, keepdim=True)[1]
    output_tensor = torch.squeeze(output_tensor, dim=0)
    output_tensor = torch.squeeze(output_tensor, dim=0)

    # 0: 배경
    # 1: 상의
    # 2: 하의
    # 3: 원피스
    output_arr = output_tensor.cpu().numpy()
    output_arr = np.where(output_arr != 0, masked_img, 0)
    output_arr = np.transpose(output_arr, (1, 2, 0))
    output_img = Image.fromarray((output_arr*255).astype('uint8'), mode='RGB')

    ############################################################
    file_name = 'Spring Couture Collection' + year + '_' + next(cnt)
    output_img.save(os.path.join(result_dir, file_name + '.jpg'))

----checkpoints loaded from path: cloth_segm_u2net_latest.pth----


100%|██████████| 23/23 [18:51<00:00, 49.18s/it]


'/content/drive/MyDrive/Colab Notebooks/AiffelThon/output/chanel'

In [ ]:
!mkdir -p '/content/drive/MyDrive/Colab Notebooks/AiffelThon/output/Christian Dior/'

In [ ]:
%%capture
!rm -rf '/content/drive/MyDrive/Colab Notebooks/AiffelThon/output/Christian Dior/Spring Couture Collection_Christian Dior_1.zip'
!zip -r '/content/drive/MyDrive/Colab Notebooks/AiffelThon/output/Christian Dior/Spring Couture Collection_Christian Dior_1.zip' output_images/